In [75]:
# importing required libraries
#!pip install --upgrade category_encoders
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
import seaborn as sns
from collections import Counter
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import roc_curve 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [76]:
# read the  data set
df = pd.read_csv('D:/ADS_APRIL/banking_churn.csv')
# top rows of the data
df.head(100)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,15699461,Fiorentini,515,Spain,Male,35,10,176273.95,1,0,1,121277.78,0
96,97,15738721,Graham,773,Spain,Male,41,9,102827.44,1,0,1,64595.25,0
97,98,15693683,Yuille,814,Germany,Male,29,8,97086.40,2,1,1,197276.13,0
98,99,15604348,Allard,710,Spain,Male,22,8,0.00,2,0,0,99645.04,0


In [77]:
# data types
print("The data set contains the following data types : \n" + str(df.dtypes) )


The data set contains the following data types : 
RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object


In [78]:
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [79]:
#Count of records
print("The data set contains the following No of records : \n" + str(df.count()) )
df.isna().sum()

The data set contains the following No of records : 
RowNumber          10000
CustomerId         10000
Surname            10000
CreditScore        10000
Geography          10000
Gender             10000
Age                10000
Tenure             10000
Balance            10000
NumOfProducts      10000
HasCrCard          10000
IsActiveMember     10000
EstimatedSalary    10000
Exited             10000
dtype: int64


RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [80]:
# Let's remove the features which has no impact on churn of the customers.
drop_list = [
'RowNumber',
'CustomerId',
'Surname',
'Geography'    
]

df=df.drop(drop_list,axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Female,43,2,125510.82,1,1,1,79084.10,0


In [81]:
df["Exited"].replace(to_replace="C", value=1, inplace=True)
df["Exited"].replace(to_replace="N", value=0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Gender           10000 non-null  object 
 2   Age              10000 non-null  int64  
 3   Tenure           10000 non-null  int64  
 4   Balance          10000 non-null  float64
 5   NumOfProducts    10000 non-null  int64  
 6   HasCrCard        10000 non-null  int64  
 7   IsActiveMember   10000 non-null  int64  
 8   EstimatedSalary  10000 non-null  float64
 9   Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(1)
memory usage: 781.4+ KB


In [82]:
# Outlier detection 

def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   

# detect outliers from Age, Balance, Duration
Outliers_to_drop = detect_outliers(df,2,["CreditScore","Age","Tenure", "Balance", "NumOfProducts", "EstimatedSalary"])

In [83]:
# Drop outliers
df = df.drop(Outliers_to_drop, axis = 0).reset_index(drop=True)

In [84]:
df = pd.get_dummies(df)

df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


## Data preprocessing

In [85]:
# Create X (all the feature columns)
X = df.drop("Exited", axis=1)

# Create y (the target column)
y = df["Exited"]
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0
1,608,41,1,83807.86,1,0,1,112542.58,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0
3,699,39,1,0.00,2,0,0,93826.63,1,0
4,850,43,2,125510.82,1,1,1,79084.10,1,0


In [86]:
# Split the data into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100, stratify=y)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 10), (2000, 10), (8000,), (2000,))

In [87]:
y_test.value_counts()

0    1593
1     407
Name: Exited, dtype: int64

In [88]:
X_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Female,Gender_Male
1814,579,31,2,90547.48,2,1,1,18800.13,0,1
4060,640,29,4,0.00,2,1,0,44904.26,1,0
797,488,32,3,114540.38,1,1,0,92568.07,0,1
5158,587,55,9,0.00,1,1,0,64593.07,0,1
7950,516,33,10,138847.90,1,1,1,127256.70,1,0


In [89]:
#Logistic pipeline


pipeline_lr=Pipeline([('scalar1',StandardScaler()),
('pca1',PCA(n_components=2)),
('lr_classifier',LogisticRegression())])
#DecisionTree pipeline
pipeline_dt=Pipeline([('scalar2',StandardScaler()),
('pca2',PCA(n_components=2)),
('dt_classifier',DecisionTreeClassifier(random_state=0))])


#RandomForest pipeline
pipeline_randomforest=Pipeline([('scalar3',StandardScaler()),
                                ('pca3',PCA(n_components=2)),
                                ('rf_classifier',RandomForestClassifier(random_state=0))])
                            
pipelines = [pipeline_lr, pipeline_dt, pipeline_randomforest]
best_accuracy=0.0,
best_classifier=0,
best_pipeline="",
pipe_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: 'RandomForest'}                          
                          
for pipe in pipelines:
    pipe.fit(X_train, y_train)
# iterate over the list
 

                               
#create variable for accuarcy and best model

                              # Fit the pipelines


In [90]:
for i,model in enumerate(pipelines):
    print("{} Test Accuracy: {}".format(pipe_dict[i],model.score(X_test,y_test)))

Logistic Regression Test Accuracy: 0.7965
Decision Tree Test Accuracy: 0.73
RandomForest Test Accuracy: 0.7945


In [91]:
#best classifier
for i,model in enumerate(pipelines):
    if model.score(X_test,y_test)>best_accuracy:
        best_accuracy=model.score(X_test,y_test)
        best_pipeline=model
        best_classifier=i
        print('Classifier with best accuracy:{}'.format(pipe_dict[best_classifier]))

Classifier with best accuracy:Logistic Regression


In [92]:
np.random.seed(42)
from sklearn.model_selection import GridSearchCV

# Define the parameters to search over
param_grid = {'n_estimators': [i for i in range(10, 101, 10)], 'max_depth': [0,5,10]}

# Setup the grid search
grid = GridSearchCV(RandomForestClassifier(),
                    param_grid,
                    cv=5,
                    scoring='recall'
                   )

# Fit the grid search to the data
grid.fit(X, y)

# Find the best parameters
grid.best_params_

C:\Users\USER\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
50 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\USER\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\Users\USER\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\USER\anaconda3\lib\site-packages\joblib

{'max_depth': 10, 'n_estimators': 20}

In [93]:
# Set the model to be the best estimator
clf = grid.best_estimator_
clf

RandomForestClassifier(max_depth=10, n_estimators=20)

In [94]:
# Fit the best model
clf = clf.fit(X_train, y_train)
# Find the best model scores
clf.score(X_test, y_test)

0.849

In [95]:
# In order to predict a label, data has to be in the same shape as X_train
X_test.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Female,Gender_Male
5779,684,40,2,70291.02,1,1,1,115468.84,0,1
9132,635,33,5,0.00,2,1,0,122949.71,1,0
992,752,42,3,65046.08,2,0,1,140139.28,1,0
8622,706,36,9,58571.18,2,1,0,40774.01,0,1
6023,786,33,0,83036.05,1,0,1,154990.58,1,0


In [96]:
import pickle
# Save an existing model to file
#pickle.dump(model, open("D:/ADS_APRIL/Assigment_2/model_1.pkl", "wb"))

In [97]:
# save the model to disk
filename = 'D:/ADS_APRIL/Assigment_2/model_2.pkl'
pickle.dump(model, open(filename, 'wb'))
 

# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, y_test)
#print(result)

In [98]:
def predict_data(X_test):
    loaded_model = pickle.load(open("D:/ADS_APRIL/Assigment_2/model_2.pkl", "rb"))
    y_test_pred = loaded_model.predict(X_test)
    return y_test_pred   

In [99]:
predict_data(X_test)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [100]:
import joblib
filename='model_2'
joblib.dump(model, filename)

['model_2']

In [101]:
Assign_2 = joblib.load(filename)
Assign_2.score(X_test, y_test)

0.7945